<a href="https://colab.research.google.com/github/dylanbforde/ai-medical-image-analysis/blob/main/MRNET_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install torchsample
!pip install visdom
!pip install nibabel
!pip install h5py
!pip install tensorboardX
!pip install optuna

In [ ]:
import optuna
import torch.optim as optim
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import os
import sys
import pickle
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from torch.autograd import Variable
from torchvision import transforms
from tensorboardX import SummaryWriter
import math
from sklearn import metrics


In [ ]:
# Modify the Net class to use R2Plus1D_18
import torchvision.models.video as video_models

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Load the pretrained R2Plus1D_18 model
        self.pretrained_model = video_models.r2plus1d_18(pretrained=True)

        # Modify the last fully connected layer to output (1)
        # The in_features for R2Plus1D_18's fc layer is 512
        self.classifer = nn.Linear(self.pretrained_model.fc.in_features, 1)

        # Add a Dropout layer after the classifier
        self.dropout = nn.Dropout(0.5) # Experiment with a dropout rate of 0.5

        # Remove the original fully connected layer from the pretrained model
        self.pretrained_model.fc = nn.Identity()


    def forward(self, x):
        # input size of x (batch_size, s, 3, 256, 256) where s is the number of slices in one MRI
        # R2Plus1D_18 expects input in (batch_size, channels, time, height, width) format
        # Permute the input to match the expected format (batch_size, 3, s, 256, 256)
        x = x.permute(0, 2, 1, 3, 4) # output size (batch_size, 3, num_slices, 256, 256)

        x = self.pretrained_model(x) # output size (batch_size, 512)

        # Pass through classifier
        output = self.classifer(x) # output size (batch_size, 1)

        # Apply dropout
        output = self.dropout(output)

        return output

# Re-initialize the model with the new architecture
model = Net()
if torch.cuda.is_available():
    model = model.cuda()

# Keep the existing optimizer, scheduler, and data loaders
# (These are defined in the previous cells and will be used in the training loop)

print("Model architecture modified to use R2Plus1D_18 model.")
print(model)

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, task, plane, train=False, transform=None):
        super().__init__()
        self.task = task
        self.plane = plane
        self.root_dir = root_dir
        self.train=train
        if self.train == True:
            self.folder_path = self.root_dir + 'train/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'train-{0}.csv'.format(task), header=None, names=['id', 'label'])
        else:
            self.folder_path = self.root_dir + 'valid/{0}/'.format(plane)

            self.records = pd.read_csv(
                self.root_dir + 'valid-{0}.csv'.format(task), header=None, names=['id', 'label'])

        self.records['id'] = self.records['id'].map(
            lambda i: '0' * (4 - len(str(i))) + str(i))
        self.paths = [self.folder_path + filename +
                      '.npy' for filename in self.records['id'].tolist()]
        self.labels = self.records['label'].tolist()

        self.transform = transform

        pos = np.sum(self.labels)
        neg = len(self.labels) - pos
        self.weights = [1, neg / pos]


    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        array = np.load(self.paths[index])

        label = self.labels[index]
        label = torch.FloatTensor([label])

        if self.transform:
          transformed_slices = []
          for i in array:
            # Apply transform to each slice
            transformed_slice = self.transform(i)
            transformed_slice = transformed_slice.repeat(3, 1, 1)
            transformed_slices.append(transformed_slice)
          array = torch.stack(transformed_slices)
        else:
          array = torch.from_numpy(array).float()
          array = array.unsqueeze(1)
          array = array.repeat(1, 3, 1, 1)

        array = array.float()

        if label.item() == 1:
            weight = np.array([self.weights[1]])
            weight = torch.FloatTensor(weight)
        else:
            weight = np.array([self.weights[0]])
            weight = torch.FloatTensor(weight)

        if array.shape[0] < 32:
            zeros = torch.zeros((32 - array.shape[0], 3, 256, 256))
            array = torch.cat((array, zeros), 0)

        elif array.shape[0] > 32:
            array = array[:32, :, :, :]

        return array, label, weight

In [ ]:
def objective(trial):
    # Define the hyperparameters to tune
    lr = trial.suggest_float('lr', 1e-6, 1e-4, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32]) # Example batch sizes
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.6) # Tune dropout rate
    trial_number = trial.number

    # --- Model Initialization ---
    model = Net()
    # Update the dropout rate in the model with the suggested value from Optuna
    model.dropout.p = dropout_rate

    if torch.cuda.is_available():
        model = model.cuda()

    # Change the optimizer to AdamW
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=2, factor=.3, threshold=1e-4, verbose=False) # Reduced patience for tuning

    # --- Data Loading ---
    directory = "/content/drive/Shared drives/MRNet Group Assignment/MRI Data/"
    task = 'acl'
    plane = 'sagittal'

    augmentor = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomRotation(25),
        transforms.RandomAffine(degrees=0, translate=(0.11, 0.11)),
        transforms.RandomHorizontalFlip(),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)) # Added Gaussian Blur
    ])

    train_dataset = Dataset(directory, task, plane, train=True, transform=augmentor)
    valid_dataset = Dataset(directory, task, plane, train=False, transform = None)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=False)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)


    # --- Training Loop (Shortened for Tuning) ---
    num_tuning_epochs = 5 # Reduced number of epochs for faster tuning trials
    best_val_auc = 0
    train_losses = []
    val_losses = []


    for epoch in range(num_tuning_epochs):
        model.train()
        y_preds = []
        y_trues = []
        running_loss = 0.0
        for i, (image, label, weight) in enumerate(train_loader):
            optimizer.zero_grad()
            if torch.cuda.is_available():
                image = image.cuda()
                label = label.cuda()
                weight = weight.cuda()

            prediction = model.forward(image.float())
            loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            probas = torch.sigmoid(prediction)
            y_trues.extend(label.cpu().tolist())
            y_preds.extend(probas.cpu().tolist())
        train_losses.append(running_loss / len(train_loader))

        # Evaluate on validation set
        model.eval()
        y_trues_val = []
        y_preds_val = []
        running_val_loss = 0.0
        with torch.no_grad():
            for i, (image, label, weight) in enumerate(valid_loader):
                if torch.cuda.is_available():
                    image = image.cuda()
                    label = label.cuda()
                    weight = weight.cuda()

                prediction = model.forward(image.float())
                loss = torch.nn.BCEWithLogitsLoss(weight=weight)(prediction, label)
                running_val_loss += loss.item()
                probas = torch.sigmoid(prediction)
                y_trues_val.extend(label.cpu().tolist())
                y_preds_val.extend(probas.cpu().tolist())
        val_losses.append(running_val_loss / len(valid_loader))

        try:
            val_auc = metrics.roc_auc_score(y_trues_val, y_preds_val)
        except:
            val_auc = 0.5 # Handle case with only one class

        # Report intermediate objective value to Optuna
        trial.report(val_auc, epoch)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if val_auc > best_val_auc:
            best_val_auc = val_auc

    # Store training and validation losses in trial user attributes
    trial.set_user_attr("train_losses", train_losses)
    trial.set_user_attr("val_losses", val_losses)

    model_save_path = f'/content/drive/MyDrive/my_acl_sagittal_model_trial_{trial_number}.pth'

    # Save the model's state dictionary
    torch.save(model.state_dict(), model_save_path)

    return best_val_auc # Return the best validation AUC for this trial

# --- Run the Optuna study ---
# Specify the number of trials to run. Keep this small initially to manage computation cost.
n_trials = 10 # You can increase this number for a more extensive search, however my compute credits are low

print(f"Running Optuna study for {n_trials} trials...")

# Create a study object and specify the direction (maximize validation AUC)
study = optuna.create_study(direction='maximize')

# Run the optimization study
study.optimize(objective, n_trials=n_trials)

print("\nOptuna study finished.")

# Print the best trial's hyperparameters and value
print("Best trial:")
print(f"  Value: {study.best_trial.value}")
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
import matplotlib.pyplot as plt


print("Visualizing training and validation losses for each trial...")

for trial in study.trials:
    trial_number = trial.number
    train_losses = trial.user_attrs.get("train_losses")
    val_losses = trial.user_attrs.get("val_losses")

    if train_losses is not None and val_losses is not None:
        epochs = range(1, len(train_losses) + 1)

        plt.figure()
        plt.plot(epochs, train_losses, label=f'Trial {trial_number} Training Loss')
        plt.plot(epochs, val_losses, label=f'Trial {trial_number} Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Training and Validation Loss for Trial {trial_number}')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print(f"Loss data not found for Trial {trial_number}")

print("Visualization complete.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# Set the model to evaluation mode
model.eval()

y_trues = []
y_preds = []
examples = []

# Disable gradient calculation for evaluation
with torch.no_grad():
    for i, (image, label, weight) in enumerate(valid_loader):
        if torch.cuda.is_available():
            # Keep image on GPU for prediction, but will move to CPU for display later
            image_gpu = image.cuda()
        else:
            image_gpu = image

        prediction = model.forward(image_gpu.float())
        probas = torch.sigmoid(prediction)

        y_trues.extend(label.cpu().tolist())
        y_preds.extend(probas.cpu().tolist())

        # Store examples with predictions, true labels, and the image data (on CPU)
        for j in range(image.size(0)):
            examples.append({
                'predicted_proba': probas[j].item(),
                'true_label': label[j].item(),
                'image': image[j].cpu().numpy() # Store image data on CPU
            })

# Display some examples
print("Examples of Classifications:")
print("-" * 30)

# Display a few correct classifications
print("Correct Classifications:")
correct_examples = [ex for ex in examples if (ex['predicted_proba'] > 0.5 and ex['true_label'] == 1.0) or (ex['predicted_proba'] <= 0.5 and ex['true_label'] == 0.0)]
for i, ex in enumerate(correct_examples[:3]): # Display up to 3 correct examples
    print(f"Example {i+1}: Predicted Probability = {ex['predicted_proba']:.4f}, True Label = {int(ex['true_label'])}")
    img_data = ex['image'] # Shape is (num_slices, channels, height, width)
    num_slices = img_data.shape[0]
    # Display a few slices
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    slices_to_show = [0, 2, 4] # Display first, third, and fifth slices
    for k, slice_idx in enumerate(slices_to_show):
        if slice_idx < num_slices:
            # Assuming grayscale image with 3 repeated channels, take one channel
            axes[k].imshow(img_data[slice_idx, 0, :, :], cmap='gray')
            axes[k].set_title(f"Slice {slice_idx}")
            axes[k].axis('off')
    plt.show()
    print("-" * 30)


# Display a few incorrect classifications
print("Incorrect Classifications:")
incorrect_examples = [ex for ex in examples if (ex['predicted_proba'] > 0.5 and ex['true_label'] == 0.0) or (ex['predicted_proba'] <= 0.5 and ex['true_label'] == 1.0)]
for i, ex in enumerate(incorrect_examples[:3]): # Display up to 3 incorrect examples
    print(f"Example {i+1}: Predicted Probability = {ex['predicted_proba']:.4f}, True Label = {int(ex['true_label'])}")
    img_data = ex['image'] # Shape is (num_slices, channels, height, width)
    num_slices = img_data.shape[0]
    # Display a few slices
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    slices_to_show = [0, 2, 4] # Display first, third, and fifth slices
    for k, slice_idx in enumerate(slices_to_show):
         if slice_idx < num_slices:
            axes[k].imshow(img_data[slice_idx, 0, :, :], cmap='gray')
            axes[k].set_title(f"Slice {slice_idx}")
            axes[k].axis('off')
    plt.show()
    print("-" * 30)